# **Dynamic Model - Brazilian Database**

As a natural continuity, this Notebook constitutes an $\color{green}{Analytical \ Database}$ containing the Brazilian results of simulation from the dynamic model.

# **Libraries**

## **General packages**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as npl
import scipy
from numpy.linalg import matrix_rank
import scipy.optimize

In [2]:
!pip install xlrd
# Graphics
!pip install -U plotly
# Excel spreadsheet organization
!pip install nltk
# Exporting pictures with better resolutions
!pip install -U kaleido

!add-apt-repository -y ppa:cran/poppler
!apt-get update
!apt-get install -y libpoppler-cpp-dev
!apt-get install poppler-utils

     |████████████████████████████████| 20.6MB 1.3MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 79.9MB 54kB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1

In [3]:
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Images directory
import os
if not os.path.exists("images"):
    os.mkdir("images")


## **Control**

In [4]:
# Control library and dynamic system

!git clone https://github.com/python-control/python-control.git

!pip install slycot
!pip install control

Cloning into 'python-control'...
remote: Enumerating objects: 7706, done.
remote: Counting objects: 100% (585/585), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 7706 (delta 409), reused 498 (delta 366), pack-reused 7121
Receiving objects: 100% (7706/7706), 8.52 MiB | 25.58 MiB/s, done.
Resolving deltas: 100% (5719/5719), done.
     |████████████████████████████████| 1.6MB 27.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for slycot: filename=slycot-0.4.0-cp37-cp37m-linux_x86_64.whl size=1417860 sha256=9eba6fafdba9688d73a90025dcd12101eb5a735c8a607fefa93bef223d126628
  Stored in directory: /root/.cache/pip/wheels/a2/46/56/f82cbb2fd06556f4f3952a2eb2396e8fd29264fffecbaad3cf
Successfully built slycot
     |████████████████████████████████| 348kB 25.2MB/s 
  Created wheel for control: filename=control-0.9.0-py2.py3-none-any.whl size=344929 sha256=35586fb671454543d84

In [5]:
import control
from control import impulse_response, step_response

# **Utilitary Functions**

In [6]:
def graphics_plot(t, y, y_title, title):
  # Plotting the results

  fig = go.Figure()

  for i in range(len(y)):
    if i<4:
      fig.add_trace(go.Scatter(
      name=y_title[i],
      x=t,
      y=y[i],
      mode='lines',
      marker_symbol='circle',
      line=dict(width=3, dash="dash")
      ))
    else:
      fig.add_trace(go.Scatter(
      name=y_title[i],
      x=t,
      y=y[i],
      mode='lines',
      marker_symbol='circle',
      line=dict(width=3, dash="solid")
      ))
  
  fig.update_layout(
      template='xgridoff',
      xaxis=dict(showgrid=False),
      xaxis_title='Date',
      yaxis_title='Individuals',
      legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.01,
        xanchor='right',
        x=0.95
        ),
      title_text=title)

  fig.show()

# **Data**

## DataFrames

>Creating a function returning the specific DataFrame for a Brazilian State.

In [7]:
def state_df(State_name):

    # Loading data - wcota
  data_path = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'
  df = pd.read_csv(data_path, delimiter=",") 

    # Dataframe for the specific
  df_state = df[df.state == State_name].reset_index()

    # Creating new recovered column
  df_state["newRecovered"] = df_state["recovered"].diff()
  df_state.newRecovered.fillna(0, inplace=True)
  df_state.recovered.fillna(0, inplace=True)

    # Creating active cases column (Infected)
  active_infected = [df_state["totalCases"].iloc[0]]
  for nc, nr in zip(df_state["newCases"].iloc[1:], 
                    df_state["newRecovered"].iloc[1:]):
      active_infected.append(active_infected[-1] + nc - nr)
  df_state["activeCases"] = active_infected

  df_state = df_state[['date','activeCases','recovered','deaths']].iloc[:200]

  return df_state

>Example

In [8]:
state_df('MA').head()

,date,activeCases,recovered,deaths
0,2020-03-20,1.0,0.0,0
1,2020-03-21,2.0,0.0,0
2,2020-03-22,2.0,0.0,0
3,2020-03-23,8.0,0.0,0
4,2020-03-24,8.0,0.0,0


## Formatting data to process

In [9]:
#  Function formatting the data to process

def data_formatting(df_state):

  Infected = np.array(df_state['activeCases']/100)
  Recovered = np.array(df_state['recovered']/100)
  Deceased = np.array(df_state['deaths']/100)
  N = Infected[-1]+Recovered[-1]+Deceased[-1]
  Susceptible = np.array(N - Infected - Recovered)
  Date = np.array(df_state['date'])

  return Date, [Susceptible,Infected,Recovered,Deceased]

>Example

In [10]:
example = data_formatting(state_df('MA'))
graphics_plot(example[0], example[1], ['Susceptible','Infected', 'Recovered', 'Deceased'], 'Covid-19 - Maranhao State')

# **Tuning the coefficients**

In [11]:
# Modification of the model - generalization

def dynamicmodel(t, x, u, parameters):
  """ Creation of the non linear model phenomenon
    t: timeline
    x: initial state vector
    u: input
    parameters: model's parameters """

  # Initial Parameters
  beta = parameters.get('beta')
  gamma = parameters.get('gamma')
  mu = parameters.get('mu')
  u_0 = 0

  # Initial Values of SIRD - x_o =[xo1 xo2].T
  S = x[0];
  I = x[1];
  R = x[2];
  D = x[3];

  # System's entry - Add content in the system - Never remove
  u = u_0 if u > 0 else 0 

  # Formalize equations
  dS = -beta*I*S
  dI = beta*I*S -gamma*I -mu*I
  dR = gamma*I
  dD = mu*I

  return [dS,dI,dR,dD]

In [12]:
# Creating the corresponding sub system
# Syntax: io_sys = NonlinearIOSystem(updfcn, outfcn, inputs=M, outputs=P, states=N)

io_model = control.NonlinearIOSystem(
    dynamicmodel, None, inputs=('u'), outputs=('S', 'I', 'R', 'D'),
    states=('S', 'I', 'R', 'D'), params={'beta':0.21691681,'gamma':0.09964236,'mu':0.00286476}, name='SIRD_model')

In [18]:
# Creating the corresponding sub system
# Syntax: io_sys = NonlinearIOSystem(updfcn, outfcn, inputs=M, outputs=P, states=N)

io_model = control.NonlinearIOSystem(
    dynamicmodel, None, inputs=('u'), outputs=('S', 'I', 'R', 'D'),
    states=('S', 'I', 'R', 'D'), params={'beta':2.84582387e-03,'gamma':6.38087033e-02,'mu':3.87843193e-03}, name='SIRD_model')

In [13]:
#  Determination of the equilibrium point

x0 = [example[1][0][0],example[1][1][0],example[1][2][0],example[1][3][0]]
equilibrium = control.find_eqpt(io_model,x0,0)
equilibrium

(array([ 8.97986520e+02, -3.66820321e-29,  1.55547993e+03,  1.08085269e+02]),
 array([0.]))

In [19]:
Sn = example[1][0]
In = example[1][1]
Rn = example[1][2]
Dn = example[1][3]
tn = np.linspace(0,len(example[0])-1,len(example[0]))

In [20]:
# Determine the merit function
def ErroQuadratico(Sn,In,Rn,Dn,tn,x0,u,parameters):
    """ function to pass to scipy.optimize.fmin
        The routine will square and sum the values returned by 
        this function""" 
    t, y = control.input_output_response(io_model, tn, u, x0, parameters)
    erroS = y[0] - Sn
    erroI = y[1] - In
    erroR = y[2] - Rn
    erroD = y[3] - Dn
    EQ = np.sum((np.concatenate([erroS,erroI,erroR,erroD])**2))
    return EQ

def objetivo(p):
  parameters = {'beta':p[0],'gamma':p[1],'mu':p[2]}
  xi = [p[3],p[4],p[5],p[6]]
  return ErroQuadratico(Sn,In,Rn,Dn,tn,xi,0,parameters)

In [ ]:
xopt = scipy.optimize.fmin(func=objetivo, x0=[0.21691681, 0.09964236, 0.00286476, 8.97986520e+02, 3.66820321e-29, 1.55547993e+03, 1.08085269e+02])

In [ ]:
xopt

array([2.84582387e-03, 6.38087033e-02, 3.87843193e-03, 1.41804399e+03,
       5.70376057e-29, 1.54443404e+02, 1.56878420e+02])

In [21]:
#  parameters = {'beta':xopt[0],'gamma':xopt[1],'mu':xopt[2]}
parameters = {'beta':2.84582387e-03,'gamma':6.38087033e-02,'mu':3.87843193e-03}


t, y = control.input_output_response(io_model, tn, 0, [1.41804399e+03, 5.70376057e-29, 1.54443404e+02, 1.56878420e+02], parameters)

#Plot
graphics_plot(example[0], [y[0],y[1],y[2],y[3],Sn,In,Rn,Dn], ['Susceptible - model','Infected - model', 'Recovered - model', 'Deceased - model','Susceptible - data','Infected - data', 'Recovered - data', 'Deceased - data'], 'Covid-19 Dynamic Modelization')

In [ ]:
def dynamic_fit(State_name):

  #Formatting State data
  DataFrame = state_df(State_name)
  npdata = data_formatting(DataFrame)

  Sn = npdata[1][0]
  In = npdata[1][1]
  Rn = npdata[1][2]
  Dn = npdata[1][3]
  tn = np.linspace(0,len(npdata[0])-1,len(npdata[0]))

  #Tune optimized parameters
  xopt = scipy.optimize.fmin(func=objetivo, x0=[0.13099162 , 0.03978684, 0.00244559,Sn[0], 3, 4.56702770, 5.58519759])

  parameters = {'beta':xopt[0],'gamma':xopt[1],'mu':xopt[2]}

  t, y = control.input_output_response(io_model, tn, 0, xopt[3:], parameters)

  #Plot
  graphics_plot(npdata[0], [y[0],y[1],y[2],y[3],Sn,In,Rn,Dn], ['Susceptible - model','Infected - model', 'Recovered - model', 'Deceased - model','Susceptible - data','Infected - data', 'Recovered - data', 'Deceased - data'], 'Covid-19 Dynamic Modelization')

  return xopt

In [ ]:
dynamic_fit('MA')

IndexError: ignored